# Setup

This notebook will guide you through the setup process for the demo. The setup
consists of the following steps:

1. **Create Kubernetes Secrets**: These secrets will allow you to download
   models from the Hugging Face Hub and NVIDIA NGC, as well as pull the
   necessary images from NVIDIA NGC.
1. **Create a Persistent Volume Claim (PVC)**: This will provide storage for the
   models.
1. **Run Kubernetes Jobs**: These jobs will download the models to the PVC.

Let's begin by importing the necessary libraries.

In [ ]:
import base64
import getpass
import subprocess

# Create the Kubernetes Secrets

The following code will create the necessary Kubernetes secrets to download the
models and images. Ensure you have your own API keys for the Hugging Face Hub
and NVIDIA NGC ready.

In [ ]:
NGC_API_KEY = getpass.getpass("Enter your NGC API key: ")

In [ ]:
HF_API_KEY = getpass.getpass("Enter your Hugging Face API key: ")

You are now ready to create the following secrets:

- **NGC CLI Secret**: This secret will store the NGC API Key, which you will use
  to download the Embeddings model.
- **Hugging Face CLI Secret**: This secret will store the Hugging Face API Key,
  which you will use to download the LLM.
- **Image Pull Secret**: This secret will enable you to download the
  NVIDIA-specific containers needed throughout this tutorial.

In [ ]:
ngc_cli_secret = """
apiVersion: v1
kind: Secret
metadata:
  name: ngc-cli-secret
type: Opaque
data:
  NGC_CLI_API_KEY: {0}
""".format(base64.b64encode(NGC_API_KEY.encode()).decode())

with open("ngc-cli-secret.yaml", "w") as f:
    f.write(ngc_cli_secret)

subprocess.run(["kubectl", "apply", "-f", "ngc-cli-secret.yaml"])

In [ ]:
hf_cli_secret = """apiVersion: v1
kind: Secret
metadata:
  name: hf-cli-secret
type: Opaque
data:
  HF_CLI_API_KEY: {0}
""".format(base64.b64encode(HF_API_KEY.encode()).decode())

with open("hf-cli-secret.yaml", "w") as f:
    f.write(hf_cli_secret)

subprocess.run(["kubectl", "apply", "-f", "hf-cli-secret.yaml"])

In [ ]:
NVCR_SECRET = """
{{"auths":{{"nvcr.io":{{"username":"$oauthtoken","password":"{0}"}}}}}}
"""

ngc_secret = """apiVersion: v1
kind: Secret
metadata:
  name: ngc-secret
type: kubernetes.io/dockerconfigjson
data:
  .dockerconfigjson: {0}
""".format(base64.b64encode(NVCR_SECRET.format(NGC_API_KEY).encode()).decode())

with open("ngc-secret.yaml", "w") as f:
    f.write(ngc_secret)

subprocess.run(["kubectl", "apply", "-f", "ngc-secret.yaml"])

# Create a Persistent Volume Claim (PVC)

Next, you will create a Persistent Volume Claim (PVC) to store the models you
download. This PVC will be mounted to the containers used in this tutorial.

In [ ]:
model_repository = """
apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: model-repository
spec:
  accessModes:
  - ReadWriteMany
  resources:
    requests:
      storage: 500Gi
  storageClassName: dataplatform
"""

with open("model-repository.yaml", "w") as f:
    f.write(model_repository)

subprocess.run(["kubectl", "apply", "-f", "model-repository.yaml"])

# Download the Models

Finally, you will download the two models you need for this tutorial:

- **Embeddings Model**: This model will embed user queries, enabling the agent
  to retrieve the most relevant information from the vector store.
- **Large Language Model (LLM)**: This model will support the agent's
  decision-making processes and generate responses for the chatbot.

Running the following Kubernetes Jobs will download the models to the PVC
automatically.

In [ ]:
ngc_downloader_job = """apiVersion: batch/v1
kind: Job
metadata:
  name: ngc-downloader
spec:
  template:
    spec:
      containers:
      - name: ngc-downloader
        image: nvcr.io/ohlfw0olaadg/ea-participants/ngc-cli:v3.41.2
        command:
        - "/bin/sh"
        - "-c"
        - |
          echo "Creating directory /mnt/model-repo/model-store"
          mkdir -p /mnt/model-repo/model-store
          echo "Downloading embeddings model..."
          ngc registry model download-version --dest /mnt/model-repo/model-store ohlfw0olaadg/ea-participants/nv-embed-qa:4
          echo "Model downloaded successfully."
        volumeMounts:
        - name: model-volume
          mountPath: /mnt/model-repo
        env:
        - name: NGC_CLI_API_KEY
          valueFrom:
            secretKeyRef:
              name: ngc-cli-secret
              key: NGC_CLI_API_KEY
        - name: NGC_CLI_ORG
          value: "nemo-microservice (ohlfw0olaadg)"
        securityContext:
          runAsUser: 0
      restartPolicy: Never
      imagePullSecrets:
      - name: ngc-secret
      volumes:
      - name: model-volume
        persistentVolumeClaim:
          claimName: model-repository
"""

with open("ngc-downloader-job.yaml", "w") as f:
    f.write(ngc_downloader_job)

subprocess.run(["kubectl", "apply", "-f", "ngc-downloader-job.yaml"])

In [ ]:
hf_downloader_job = """apiVersion: batch/v1
kind: Job
metadata:
  name: hf-downloader
spec:
  template:
    spec:
      containers:
      - name: hf-downloader
        image: python:3.12.3
        command:
        - "/bin/sh"
        - "-c"
        - |
          echo "Installing the HuggingFace CLI..."
          pip install -U huggingface_hub[cli]
          echo "Creating directory /mnt/model-repo/Llama-2-7b-chat-hf"
          mkdir -p /mnt/model-repo/Llama-2-7b-chat-hf
          echo "Downloading Llama 2 7B Chat from HuggingFace HUB..."
          huggingface-cli download meta-llama/Llama-2-7b-chat-hf --local-dir /mnt/model-repo/Llama-2-7b-chat-hf
          echo "Model downloaded successfully."
        volumeMounts:
        - name: model-volume
          mountPath: /mnt/model-repo
        env:
        - name: HF_TOKEN
          valueFrom:
            secretKeyRef:
              name: hf-cli-secret
              key: HF_CLI_API_KEY
        securityContext:
          runAsUser: 0
      restartPolicy: Never
      volumes:
      - name: model-volume
        persistentVolumeClaim:
          claimName: model-repository
"""

with open("hf-downloader-job.yaml", "w") as f:
    f.write(hf_downloader_job)

subprocess.run(["kubectl", "apply", "-f", "hf-downloader-job.yaml"])

# Conclusion

You are now ready to deploy the demo. Use the provided Helm chart to install the
application using the **Import Framework** wizard in the Unified Analytics
Platform.